In [ ]:
## This is the answer guide notebook 
## It has the un-modified data and good hyperparams

In [26]:
# TODO Get this made public

! pip install git+https://github.com/deepskies/DeepBench.git
! python3 make_data.py


  Cloning https://github.com/deepskies/DeepBench.git (to revision bug/import) to /private/var/folders/95/_pswfcf9723cb9f4vw6h944m0000gr/T/pip-req-build-gt72anhg
  Running command git clone --filter=blob:none --quiet https://github.com/deepskies/DeepBench.git /private/var/folders/95/_pswfcf9723cb9f4vw6h944m0000gr/T/pip-req-build-gt72anhg
  Running command git checkout -b bug/import --track origin/bug/import
  Switched to a new branch 'bug/import'
  branch 'bug/import' set up to track 'origin/bug/import'.
  Resolved https://github.com/deepskies/DeepBench.git to commit 2718040cd0fd0d51b78146f42cbe684d70f103b1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for deepbench: filename=deepbench-0.1.1-py3-none-any.whl size=29920 sha256=ce11d8beb7c05e43dbb220d1a7762fee9ae23f18942247874ac0443cc11d91f4
  Stored in directory: /private/var/folders/95/_pswfcf9723cb9f4vw6h944m0000gr/T/pip-ephem-wheel

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Dropout, Conv2D, Dense



In [17]:
# don't worry too much about how this simulation is being run. It's fine. Don't worry. I promise :3c 
# Loading the generated data with a 


ModuleNotFoundError: No module named 'deepbench.src.deepbench.physics_object'

In [ ]:
split_index = 

training_features = []
validation_features = []

training_labels = []
validation_labels = []

In [ ]:
train_generator = {}
validation_generator = {}

In [ ]:
model = {}

In [ ]:
model.fit()

In [ ]:
# Plot history

In [ ]:
# Making a different testing set (So people can see the in-domain vs out-of-domain)
testing_set = {}
testing_generator = {}

In [ ]:
# Metrics and evaluation